# 存储大型矩阵
求解大型矩阵的结果以便于后续的后处理，这里对比scipy和h5py的存储效率，包括时间和文件大小

实验结果如下

|                  |      | 随机矩阵 | (10000, 10000) |     | 稀疏矩阵 | (10000, 10000) |
|:----------------:|:----:|:--------:|:--------------:| --- |:--------:|:--------------:|
|      库函数      | 选项 |  耗时/s  |  文件大小/MB   |     |  耗时/s  |  文件大小/MB   |
|     np.save      |      |   0.648    |      763       |     |  0.175   |     228 K      |
| scipy.io.savemat | 默认 |   2.39   |      763       |     |  0.143   |     160 K      |
|                  | 压缩 |    63    |      720       |     |  0.140   |      76 K      |
|       h5py       | 默认 |   1.26   |      763       |     |   1.33   |      763       |
|                  | gzip |   55.1   |      720       |     |   7.7    |      1.4       |
|                  | lzf  |   8.02   |      764       |     |   1.98   |      9.3       |
|                  | szip |    19    |      698       |     |   5.9    |       11       |

结论:**numpy最好，h5py次好**

1. 对于随机矩阵，两者文件基本相同，但h5py效率更好
2. 对于稀疏矩阵，scipy.io.savemat效率更好而且文件更小
3. 因此h5py更适合用于密集矩阵，而scipy比较适合稀疏矩阵。**但综合考虑来看numpy最好，h5py次好（密集矩阵更常见）**

## Exp. 1 大型随机矩阵

In [1]:
# 建立大型随机矩阵
import numpy as np
import os
from scipy.io import loadmat
A = np.random.rand(10000, 10000)
print('随机矩阵 A:', A.ndim, A.shape, A.dtype)

随机矩阵 A: 2 (10000, 10000) float64


In [2]:
from scipy.io import savemat
import h5py

### a. numpy方法

In [3]:
%time np.save('npfile',A)
os.system('du -sh npfile.npy')
%time np.savez('npfile',A)
os.system('du -sh npfile.npz')

CPU times: user 0 ns, sys: 650 ms, total: 650 ms
Wall time: 648 ms
763M	npfile.npy
CPU times: user 1.31 s, sys: 836 ms, total: 2.15 s
Wall time: 2.48 s
763M	npfile.npz


0

### b. scipy.io.savemat方法
通过控制结果发现压缩之后时间增加但文件变小选项，这里我们对比了压缩与不压缩的区别，通过测试发现：压缩之后时间增加但文件变小

In [4]:
# 随机矩阵
print('1）scipy.io.savemat默认选项：')
%time savemat('matfile.mat', {'elem':A})
os.system('du -sh matfile.mat')
print('\n2）scipy.io.savemat压缩选项：')
%time savemat('matfile_com.mat', {'elem':A}, do_compression=True)
os.system('du -sh matfile_com.mat')

1）scipy.io.savemat默认选项：
CPU times: user 949 ms, sys: 1.17 s, total: 2.12 s
Wall time: 2.39 s
763M	matfile.mat

2）scipy.io.savemat压缩选项：
CPU times: user 1min 2s, sys: 1.85 s, total: 1min 4s
Wall time: 1min 5s
720M	matfile_com.mat


0

### c. h5py 方法
同样我们对比了压缩与不压缩的区别，通过测试发现：

In [5]:
# 默认格式
print('1）h5py默认选项：')
%time with h5py.File('h5file.h5', 'w') as hf:  hf.create_dataset('elem', data=A)
os.system('du -sh h5file.h5')

print('\n2）h5py压缩选项：')
# 压缩选项 'gzip'
%time with h5py.File('h5file_com_gzip.h5', 'w') as hf:    hf.create_dataset('elem', data=A, compression='gzip')
os.system('du -sh h5file_com_gzip.h5')
# 压缩选项 'lzf'
%time with h5py.File('h5file_com_lzf.h5', 'w') as hf:    hf.create_dataset('elem', data=A, compression='lzf')
os.system('du -sh h5file_com_lzf.h5')
# 压缩选项 'szip'
%time with h5py.File('h5file_com_szip.h5', 'w') as hf:    hf.create_dataset('elem', data=A, compression='szip')
os.system('du -sh h5file_com_szip.h5')

1）h5py默认选项：
CPU times: user 1.02 ms, sys: 1.12 s, total: 1.12 s
Wall time: 1.37 s
763M	h5file.h5

2）h5py压缩选项：
CPU times: user 55.3 s, sys: 764 ms, total: 56.1 s
Wall time: 56.2 s
720M	h5file_com_gzip.h5
CPU times: user 7.17 s, sys: 756 ms, total: 7.93 s
Wall time: 8.27 s
764M	h5file_com_lzf.h5
CPU times: user 17.8 s, sys: 688 ms, total: 18.5 s
Wall time: 18.9 s
698M	h5file_com_szip.h5


0

## Exp. 2 大型稀疏矩阵

In [6]:
# 建立大型稀疏矩阵
from scipy.sparse import lil_matrix
L=lil_matrix((10000,10000))
L[100:200,100:200]=np.random.rand(100, 100)
print('稀疏矩阵 L:', L.shape, L.dtype)

稀疏矩阵 L: (10000, 10000) float64


### a. numpy方法

In [7]:
%time np.save('npfile',L)
os.system('du -sh npfile.npy')

%time np.savez('npfile',L)
os.system('du -sh npfile.npz')

CPU times: user 1.4 ms, sys: 160 ms, total: 161 ms
Wall time: 162 ms
228K	npfile.npy
CPU times: user 3.33 ms, sys: 152 ms, total: 155 ms
Wall time: 155 ms
228K	npfile.npz


0

### b. scipy.io.savemat方法
通过控制do_compression选项，这里我们对比了压缩与不压缩的区别，通过测试发现：

In [8]:
print('1）scipy.io.savemat默认选项：')
%time savemat('matfile.mat', {'elem':L})
os.system('du -sh matfile.mat')
print('\n2）scipy.io.savemat压缩选项：')
%time savemat('matfile_com.mat', {'elem':L}, do_compression=True)
os.system('du -sh matfile_com.mat')

1）scipy.io.savemat默认选项：
CPU times: user 3.36 ms, sys: 152 ms, total: 156 ms
Wall time: 153 ms
160K	matfile.mat

2）scipy.io.savemat压缩选项：
CPU times: user 9.02 ms, sys: 136 ms, total: 145 ms
Wall time: 145 ms
76K	matfile_com.mat


0

### c. h5py 方法
同样我们对比了压缩与不压缩的区别，但是注意需要把稀疏矩阵转化为密集矩阵，通过测试发现：

In [9]:
# 默认格式
print('1）h5py默认选项：')
%time with h5py.File('h5file.h5', 'w') as hf:  hf.create_dataset('elem', data=L.todense())
os.system('du -sh h5file.h5')

print('\n2）h5py压缩选项：')
# 压缩选项 'gzip'
%time with h5py.File('h5file_com_gzip.h5', 'w') as hf:    hf.create_dataset('elem', data=L.todense(), compression='gzip')
os.system('du -sh h5file_com_gzip.h5')
# 压缩选项 'lzf'
%time with h5py.File('h5file_com_lzf.h5', 'w') as hf:    hf.create_dataset('elem', data=L.todense(), compression='lzf')
os.system('du -sh h5file_com_lzf.h5')
# 压缩选项 'szip'
%time with h5py.File('h5file_com_szip.h5', 'w') as hf:    hf.create_dataset('elem', data=L.todense(), compression='szip')
os.system('du -sh h5file_com_szip.h5')

1）h5py默认选项：
CPU times: user 12.2 ms, sys: 1.08 s, total: 1.09 s
Wall time: 1.36 s
763M	h5file.h5

2）h5py压缩选项：
CPU times: user 7.29 s, sys: 344 ms, total: 7.63 s
Wall time: 7.68 s
1,4M	h5file_com_gzip.h5
CPU times: user 1.6 s, sys: 328 ms, total: 1.93 s
Wall time: 1.94 s
9,3M	h5file_com_lzf.h5
CPU times: user 5.5 s, sys: 336 ms, total: 5.84 s
Wall time: 5.92 s
11M	h5file_com_szip.h5


0